# Playground One Shot Learning

In [ ]:
COLAB = False
if COLAB : 
    !pip install tensorflow-addons
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
import numpy as np
import shutil
import os
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from plotly import express as px
import cv2

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras import optimizers

import logging
logging.getLogger('tensorflow_addons').setLevel(logging.ERROR)

In [ ]:
if COLAB : 
    DATA_PATH = '/content/gdrive/MyDrive/Projects/BleauGuessr/data/butteauxpeintres2/'
else :
    DATA_PATH = '../data/butteauxpeintres2/'

folders = next(os.walk(DATA_PATH))[1]  # Get a list of all subdirectories
NUM_CLASSES = len(folders)  # Count the number of subdirectories

IMG_SIZE = (256, 256)

In [ ]:
n_boulders = 0
n_imgs = 0
for boulder in folders : 
    n_boulders += 1
    n_imgs += len(os.listdir(DATA_PATH+boulder))
print('Total boulders :', n_boulders)
print('Total images :', n_imgs)

In [ ]:
X = []
Y = []
n_boulder = 0
cat_dict = {}
for boulder in folders : 
    cat_dict[n_boulder] = boulder
    boulder_folder = DATA_PATH + boulder
    for img_path in os.listdir(boulder_folder) :
        img_path = os.path.join(boulder_folder, img_path)
        img = tf.keras.utils.load_img(img_path)
        img = tf.image.resize(img, (256,256)) / 255.
        X.append(img)
        Y.append(n_boulder)
    n_boulder+=1
X = np.array(X) # (56, 256, 256, 3)
Y = np.array(Y) # (56, 1)
cat_dict

In [ ]:
def get_batch(batch_size, X=X, Y=Y) :

    X = X.copy()
    Y = Y.copy() 

    n_classes = len(cat_dict)

    # (right-side) boulders selected for this batch
    boulders_classes = np.random.choice(n_classes, size=(batch_size,), replace=False) # example : [0,8,1,5,10,9]

    # Initialize pairs of images
    pairs = [np.zeros((batch_size, 256, 256 , 3)) for i in range(2)] # list of 2 : [(6, 256, 265, 3), (6, 256, 265, 3)]

    # Initialiaz targets
    targets = np.random.randint(2, size=6) # example : [0,1,1,0,1,0]

    for i in range(batch_size) : 
        
        class_i = boulders_classes[i]

        # Take random img from this class
        list_available_imgs = np.argwhere(Y == class_i).T[0]

        n_img_1 = np.random.choice(list_available_imgs)

        pairs[0][i, ...] = X[n_img_1]

        # 2nd image is same class (target == 1)
        if targets[i] : 
            list_available_imgs_2 = np.setdiff1d(list_available_imgs, n_img_1)
        else : 
            class_i_2 = np.random.choice(np.setdiff1d(np.arange(n_classes), class_i))

            list_available_imgs_2 = np.argwhere(Y == class_i_2).T[0]

        n_img_2 = np.random.choice(list_available_imgs_2)

        pairs[1][i, ...] = X[n_img_2]


    return (pairs, targets)

In [ ]:
def batch_generator():
    """
    a generator for batches, so model.fit_generator can be used.
    """
    batch_size=6
    while True:
        pairs, targets = get_batch(batch_size)
        yield (pairs, targets)

In [ ]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(32, (5,5), activation='relu', input_shape=input_shape,))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (3,3), activation='relu',))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (3,3), activation='relu',))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (3,3), activation='relu',))
    model.add(MaxPooling2D())
    model.add(Conv2D(512, (3,3), activation='relu',))
    model.add(Flatten())
    model.add(Dense(512, activation='sigmoid',))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors : tf.keras.backend.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net


In [ ]:
model = get_siamese_model((256, 256, 3))
model.compile(loss="binary_crossentropy",optimizer=optimizer=optimizers.Adam(0.0001))
model.summary()

In [ ]:
# Hyper parameters
n_iter = 2000 # No. of training iterations

# No testing for now which is really bad lol

print("Starting training process!")
print("-------------------------------------")
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(6)
    loss = model.train_on_batch(inputs, targets)
    print(loss)

In [ ]:
x, y = get_batch(6)
print(y)
model.predict(x)